# Representations of Finite Groups I — Lecture Examples

**6.7970/8.750 Symmetry and its Application to Machine Learning**

This notebook contains live examples from the Representations I lecture: irreps of $P(3)$, Schur's Lemma, and the Wonderful Orthogonality Theorem.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/rep1_examples.ipynb)

## Setup

In [ ]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip

In [ ]:
import numpy as np
from symm4ml import groups, linalg, rep, vis

# Representations of $P(3)$ that we've seen so far

## 2D rotations and mirrors ($D_3$, symmetry of a triangle)

In [ ]:
vis.plot_matrices(groups.p3_dresselhaus, cmap='RdBu');

## 3D Permutation matrices

In [ ]:
vis.plot_matrices(groups.p3, cmap='RdBu');

# The (Left) Regular Representation of $P(3)$

In [ ]:
p3_reg_rep = rep.regular_representation(groups.make_multiplication_table(groups.p3))
vis.plot_matrices(p3_reg_rep, cmap='RdBu');

# Schur's Lemma Part I
If $M D(g) = D(g) M, \forall g \in G$, $M$ is either a constant matrix, or $D(g)$ is <b>reducible</b>.

In [ ]:
cob = linalg.infer_change_of_basis(groups.p3_dresselhaus, groups.p3_dresselhaus)
# Irreducible!
vis.plot_matrices(cob, vmin=-1, vmax=1, figsize=(1.5, 1.5), cmap='RdBu');

In [ ]:
cob = linalg.infer_change_of_basis(groups.p3, groups.p3)
# Reducible!
vis.plot_matrices(cob, vmin=-1, vmax=1, figsize=(4, 2), cmap='RdBu');

In [ ]:
# Need to make sure elements are in the same order
table_p3 = groups.make_multiplication_table(groups.p3)
table_p3_dresselhaus = groups.make_multiplication_table(groups.p3_dresselhaus)
isos = list(groups.isomorphisms(table_p3, table_p3_dresselhaus))
# Do p3 and p3_dresselhaus share any irreps?
# Yes! They share the 2D irrep
cob = linalg.infer_change_of_basis(groups.p3, groups.p3_dresselhaus[list(isos[0])])
vis.plot_matrices(cob, vmin=-1, vmax=1, figsize=(4, 2), cmap='RdBu');

These matrices define linear space of solutions

In [ ]:
test = lambda x: x * cob[0] + (1 - x) * cob[1]
np.allclose(np.einsum('ij,njk->nik', test(0.5), groups.p3), np.einsum('nij,jk->nik', groups.p3, test(0.5)))

# Irreps of $P(3)$

$
\begin{array}{|c|c|c|c|c|c|c|}
\hline
 & D_\Gamma(E) & D_\Gamma(A) & D_\Gamma(B) & D_\Gamma(C) & D_\Gamma(D) & D_\Gamma(F)\\
\hline
\Gamma_{1} (A_1) & 1 &1 &1 & 1 & 1 &1\\
\Gamma_{1'} (A_2) & 1 & -1 &-1 & -1 & 1 &1\\
\Gamma_2 (E) & \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} & \begin{pmatrix} -1 & 0 \\ 0 & 1 \end{pmatrix} & \begin{pmatrix} 1/2 & -\sqrt{3}/2 \\ -\sqrt{3}/2  & -1/2 \end{pmatrix} & \begin{pmatrix} 1/2 & \sqrt{3}/2 \\ \sqrt{3}/2  & -1/2 \end{pmatrix} & \begin{pmatrix} -1/2 & \sqrt{3}/2 \\ -\sqrt{3}/2  & -1/2 \end{pmatrix} & \begin{pmatrix} -1/2 & -\sqrt{3}/2 \\ \sqrt{3}/2  & -1/2 \end{pmatrix} \\
\hline
\end{array}
$

# We can also use `linalg.infer_change_of_basis` to see if there are any irreps shared between two representations

## There is a copy of the 2D irrep of $P(3)$ in the permutation matrices

In [ ]:
iso = list(groups.isomorphisms(
    groups.make_multiplication_table(groups.p3),
    groups.make_multiplication_table(groups.p3_dresselhaus)
))
linalg.infer_change_of_basis(groups.p3, groups.p3_dresselhaus[list(iso[0])])

# There is a copy of the trivial i.e. invariant irrep ($\Gamma_1$) in the permutation matrices

In [ ]:
linalg.infer_change_of_basis(np.ones([6, 1, 1]), groups.p3)

## There is no copy of trivial i.e. invariant irrep ($\Gamma_1$) in the 3D permutation matrices

In [ ]:
linalg.infer_change_of_basis(np.ones([6, 1, 1]), groups.p3_dresselhaus)

# Wonderful orthogonality theorem

$\sum_R D_{\mu \nu} ^{(\Gamma_j)}(R) D_{\nu' \mu'}^{(\Gamma_{j'})}(R^{-1}) = \frac{h}{l_j}\delta_{\Gamma_j \Gamma_{j'}} \delta_{\mu \mu'} \delta_{\nu \nu'}$

If unitary representation


$\sum_R D_{\mu \nu} ^{(\Gamma_j)}(R) \left[ D_{\nu' \mu'}^{(\Gamma_{j'})}(R) \right]^* = \frac{h}{l_j}\delta_{\Gamma_j \Gamma_{j'}} \delta_{\mu \mu'} \delta_{\nu \nu'}$

## We know from trying out `linalg.infer_change_of_basis` on the 2D rotation + mirror representation of $P(3)$ that it is an irreducible representation. Let's check the Wonderful Orthogonality Theorem on the individual vectors within this representation!

In [ ]:
# Each representation vector is shape (6, 1, 1)
# Perform inner product of each vector with itself
lj = 2
h = 6
# equals h / l_j = 6 / 2 = 3
p3_ortho = np.einsum(
    'nijkl,nijkl->ijkl',
    groups.p3_dresselhaus.reshape(h, lj, lj, 1, 1),
    groups.p3_dresselhaus.reshape(h, 1, 1, lj, lj,)
)

# Let's re-write the Wonderful Orthogonality Theorem with different indices to better match what we will use in our einsum

$\sum_R D_{ij} ^{(\Gamma_a)}(R) \left[ D_{kl}^{(\Gamma_{b})}(R) \right]^* = \frac{h}{l_j}\delta_{\Gamma_a \Gamma_{b}} \delta_{i k} \delta_{jl}$

### Only nonzero, when $\delta_{ik}\delta_{jl}$

In [ ]:
i, j, k, l = p3_ortho.nonzero()
i, j, k, l

## And the nonzeros are indeed $\frac{h}{l_j} = \frac{6}{2} = 3$

In [ ]:
p3_ortho.reshape(4, 4) # rows and colums are (0, 0), (0, 1), (1, 0), (1, 1) vectors of rep